In [1]:
#Imports:
import os
import sys
import pickle
#if you want the dogtoys module, feed the path here and uncomment
sys.path.append('/Users/Owner/Documents/GitHub/narrative-recommend')
from kitchentable.dogtoys import *

from json import loads
from re import findall,UNICODE

from labMTsimple.speedy import LabMT 
my_LabMT = LabMT()   # <---- Can replace this with whatever sentiment ratings you want
from labMTsimple.storyLab import *

from spacy.lang.en import English
nlp = English()
from scipy.sparse import lil_matrix,issparse

import numpy as np
from database.bookclass import *
import pickle

In [48]:
#Cleans a book.  Taken from andy's code.
#GUTENBERG ONLY
def get_maintext_lines_gutenberg(raw_text):
    raw_text = raw_text.read()
    lines = raw_text.split("\n")
    start_book_i = 0
    end_book_i = len(lines)-1
    start_title_i = 0
    end_title_i = len(lines)-1
    end_title_best = len(lines)-1
    print("start",start_title_i, end_title_i)
    # pass 1
    title_start = "Title: "
    title_end = "Author:"
    start1="START OF THIS PROJECT GUTENBERG EBOOK"
    start2="START OF THE PROJECT GUTENBERG EBOOK"
    end1="END OF THIS PROJECT GUTENBERG EBOOK"
    end2="END OF THE PROJECT GUTENBERG EBOOK"
    end3="END OF PROJECT GUTENBERG"
    for j,line in enumerate(lines):
        if (start1 in line) or (start2 in line):
            # and "***" in line and start_book[i] == 0 and j<.25*len(lines):
            start_book_i = j
        end_in_line = end1 in line or end2 in line or end3 in line.upper()
        if end_in_line and (end_book_i == (len(lines)-1)):
            #  and "***" in line and j>.75*len(lines)
            end_book_i = j
        #below if statements capture the title from a txt file
        if (title_start in line):
            start_title_i = j
        if (title_end in line):
            #check if lower
            if (j < end_title_best):
                end_title_i = j
                end_title_best = j
        print("two", start_title_i, end_title_i)
    # pass 2, this will bring us to 99%
    if (start_book_i == 0) and (end_book_i == len(lines)-1):
        for j,line in enumerate(lines):
            if ("end" in line.lower() or "****" in line) and  "small print" in line.lower() and j<.5*len(lines):
                start_book_i = j
            if "end" in line.lower() and "project gutenberg" in line.lower() and j>.75*len(lines):
                end_book_i = j
        # pass three, caught them all (check)
        if end_book_i == len(lines)-1:
            for j,line in enumerate(lines):
                if "THE END" in line and j>.9*len(lines):
                    end_book_i = j
    print(start_title_i, end_title_i)
    print(lines[(start_title_i):(end_title_i)])
    title = lines[(start_title_i):(end_title_i)]
    return title, lines[(start_book_i+1):(end_book_i)]

In [49]:
def score_book(book_file):
    word_list = []
    timeseries = None

    # Hyperparameters for generating timeseries: play around!
    num_points = 200
    min_size = 10000
    stop_val = 2.0
    
    #get raw text
    #need to check encoding before reading
    #either utf-8 or ISO
    print(book_file)
    if "-8.txt" in book_file:
        b_data = open(book_file, "r", encoding="ISO-8859-1")
    else:
        b_data = open(book_file, "r", encoding="utf8")
    #then preprocess/clean it
    title, b_data = (get_maintext_lines_gutenberg(b_data))

    #this gets every single sentence from the book
    chunks = chunkify(b_data)
    #and this puts them back into list format
    chunks = list(map(lambda x: [x[0],x[1],nlp(x[2]),x[2]],chunks))   

    #then create word_list
    for chunk in chunks:
        for token in chunk[2]:
            #We grab each individual word from the chunk[2], which is the sentence
            #and append to word_list if it's not in there
            if (not token.is_punct) and (len(str(token)) != 0):
                word_list.append(str(token)) # get token.orth_

    step = int(floor((len(word_list)-min_size)/(num_points-1)))

    #build matrix with this function:
    def build_matrix(num_points,word_list,step,min_size,my_senti_dict):
        '''Build the matrix of overlapping word vectors.'''
        # disregard the existing matrix, it might not be the right size
        all_fvecs = lil_matrix((num_points,len(my_senti_dict.scorelist)),dtype="i")
        for i in range(num_points-1):
            all_fvecs[i,:] = my_senti_dict.wordVecify(dictify(word_list[(i*step):(min_size+i*step)]))
        i = num_points-1
        all_fvecs[i,:] = my_senti_dict.wordVecify(dictify(word_list[(i*step):]))
        return all_fvecs  #returns overlapping word vectors
        
    all_fvecs = build_matrix(num_points,word_list,step,min_size,my_LabMT).tocsr()
    
    timeseries = [0 for i in range(num_points)]
    for i in range(num_points):
        text_fvec = all_fvecs[i,:].toarray().squeeze()
        stoppedVec = stopper(text_fvec,my_LabMT.scorelist,my_LabMT.wordlist,stopVal=stop_val)  #NOTE: stopper comes from storylab MT
        timeseries[i] = dot(my_LabMT.scorelist,stoppedVec)/sum(stoppedVec)
    
    return title, timeseries

In [50]:
# One book
book_file = r"C:\Users\Owner\Documents\GitHub\New folder\books\10023.txt"
title, test = score_book(book_file)
print(title)

C:\Users\Owner\Documents\GitHub\New folder\books\10023.txt
start 0 3583
two 0 3583
two 0 3583
two 0 3583
two 0 3583
two 0 3583
two 0 3583
two 0 3583
two 0 3583
two 8 3583
two 8 3583
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10
two 8 10